In [17]:
'''샘플의 해시값 계산하기'''
import sys
import hashlib

# 해시값을 계산할 파일을 선택
filename = "./datasets/test.exe"

# MD5/SHA-256 인스턴스 로드, Chunk 크기를 지정
BUF_SIZE = 65536
md5 = hashlib.md5()
sha256 = hashlib.sha256()

# 64KB만큼을 읽어 해시값을 계산한다.
with open(filename, 'rb') as f:
    while True:
        data = f.read(BUF_SIZE)
        if not data:
            break
        md5.update(data)
        sha256.update(data)

# 최종 해시값을 16진수로 출력한다.
print("{}의 MD5 해시값: \n{}\n".format(filename, md5.hexdigest()))
print("{}의 SHA-256 해시값: \n{}\n".format(filename, sha256.hexdigest()))

./datasets/test.exe의 MD5 해시값: 
fb95de2db64beb631e865e2edb7e93e6

./datasets/test.exe의 SHA-256 해시값: 
a0a21df8603efafa3ee50e318b9fb2790eef1f66a2391b35f754c899e2f1a979



## YARA
- 규칙과 일치하는 샘플을 분류하는데 도움을 주는 도구
- 코드 예시
```YARA
# condition과 일치하지 않는 샘플 선택
rule my_rule_name { condition: false }

# condition에 일치하는 샘플 선택
rule my_rule_name { condition: true }

# 예시) 100KB가 넘는 모든 파일을 선택하기
rule over_100kb { condition: filesize > 100KB }

# 예시) PDF의 시그니처를 기반으로, PDF 파일인지 판단
rule is_a_pdf {
    strings:
        $pdf_magic = { 25 50 44 46 }
    condition:
        $pdf_magic at 0
}
```

- yara 실습
    1. 아래의 규칙을 텍스트 파일에 복사하고, 텍스트 파일의 이름을 rules.yara로 한다.
    ```YARA
    rule is_a_pdf
    {
        strings:
            $pdf_magic = { 25 50 44 46 }
        condition:
            $pdf_magic at 0
    }
    rule dummy_rule1
    {
        condition: false
    }
    rule dummy_rule2
    {
        condition: true
    }
    ```

    2. YARA 규칙을 적용할 파일을 선택하고, 다음과 같이 터미널에서 명령어를 입력한다.
    ```bash
    yara rule.yara PythonBrochure

    # 결과
    is_a_pdf PythonBrochure.pdf
    dummy_rule2 PythonBrochure.pdf
    ```


In [21]:
'''PE 헤더 검사'''
import pefile

# 분석하고자 하는 파일을 불러들인다.
desired_file = "./datasets/test.exe"
pe = pefile.PE(desired_file)

# PE 파일이 임포트 하는 항목을 나열한다.
for entry in pe.DIRECTORY_ENTRY_IMPORT:
    print(entry.dll)
    for imp in entry.imports:
        print(f"\t{hex(imp.address)},{imp.name}")

# PE 파일의 섹션을 나열한다.
for section in pe.sections:
    print(
        section.Name,
        hex(section.VirtualAddress),
        hex(section.Misc_VirtualSize),
        section.SizeOfRawData,
    )

# 구문 분석한 정보의 전체 덤프를 출력한다.
print(pe.dump_info())

b'KERNEL32.dll'
	0x1400279f8,b'SetEvent'
	0x140027a00,b'GlobalFree'
	0x140027a08,b'GetLocaleInfoW'
	0x140027a10,b'CreateFileW'
	0x140027a18,b'ReadFile'
	0x140027a20,b'GetCurrentProcess'
	0x140027a28,b'GetCommandLineW'
	0x140027a30,b'HeapSetInformation'
	0x140027a38,b'LocalFree'
	0x140027a40,b'LocalAlloc'
	0x140027a48,b'FindFirstFileW'
	0x140027a50,b'FindClose'
	0x140027a58,b'FoldStringW'
	0x140027a60,b'K32GetModuleFileNameExW'
	0x140027a68,b'GetModuleFileNameW'
	0x140027a70,b'GetUserDefaultUILanguage'
	0x140027a78,b'GetLocalTime'
	0x140027a80,b'GetDateFormatW'
	0x140027a88,b'GetTimeFormatW'
	0x140027a90,b'WideCharToMultiByte'
	0x140027a98,b'WriteFile'
	0x140027aa0,b'GetFileAttributesW'
	0x140027aa8,b'LocalLock'
	0x140027ab0,b'GetACP'
	0x140027ab8,b'CreateEventExW'
	0x140027ac0,b'DeleteFileW'
	0x140027ac8,b'SetEndOfFile'
	0x140027ad0,b'GetFileAttributesExW'
	0x140027ad8,b'GetFileInformationByHandle'
	0x140027ae0,b'CreateFileMappingW'
	0x140027ae8,b'MapViewOfFile'
	0x140027af0,b'MultiByt

In [1]:
'''PE 헤더 특성화'''
import pefile
from os import listdir
from os.path import isfile, join

directories = ["Benign PE Samples", "Malicious PE Samples"]

# 파일 섹션 이름을 수집하고 함수와 가독성과 정규화를 위해 섹션 이름을 전처리하는 함수를 정의
def get_section_names(pe):
    # PE 파일에서 섹션의 이름 리스트를 만든다
    list_of_section_names = []
    for sec in pe.sections:
        normalized_name = sec.Name.decode().replace("\x00","").lower()
        list_of_section_names.append(normalized_name)
    return list_of_section_names

# import 항목을 전처리한 후 이름을 소문자로 표준화하는 함수를 정의한다.
def preprocess_imports(list_of_DLLs):
    # PE파일에서 Import 항목의 이름을 소문자로 정규화한다.
    return [x.decode().split(".")[0].lower() for x in list_of_DLLs]

# pefile을 사용해 파일에서 import 항목을 수집하븐ㄴ 함수를 정의한다.
def get_imports(pe):
    # PE 파일에서 Import 항목들의 리스트를 만든다.
    list_of_imports = []
    for entry in pe.DIRECTORY_ENTRY_IMPORT:
        list_of_imports.appned(entry.dll)
    return preprocess_imports(list_of_imports)

# 마지막으로 모든 파일에 대해 반복하고 특성을 저장할 리스트를 만든다.
imports_corpus = []
num_sections = []
section_names = []

for dataset_path in directories:
    samples = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]
    for file in samples:
        file_path = dataset_path + "/" + file
        try:
            # 위의 특성을 수집하는 것 이외에 파일의 섹션 갯수도 수집한다.
            pe = pefile.PE(file_path)
            imports = get_imports(pe)
            n_sections = len(pe.sections)
            sec_names = get_section_names(pe)
            imports_corpus.append(imports)
            num_sections.append(n_sections)
            section_names.append(sec_names)
        except Exception as e:
            print(e)
            print(f"{file_path}의 헤더를 구분 분석할 수 없습니다.")

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'Benign PE Samples'